In [1]:
!pip install strsimpy
from strsimpy.jaro_winkler import JaroWinkler
from strsimpy.normalized_levenshtein import NormalizedLevenshtein

import pandas as pd
import numpy as np
import json
import fileinput
from tqdm import tqdm
import string
import re
import json
from os import walk
import os
import ast
import re

data_dir = "../data_/structured_data/"

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/opt/anaconda3/envs/jaeyoung/bin/python -m pip install --upgrade pip' command.


In [2]:
f = []
for (dirpath, dirnames, filenames) in walk(data_dir):
    f.extend(filenames)

In [3]:
tot_game_df = pd.DataFrame()
for file in f:
    if file[0] == "s":
        df = pd.read_csv(data_dir + file)
        tot_game_df = tot_game_df.append(df)

In [4]:
tot_game_df

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,LBA,SJH,SJD,SJA,BSH,BSD,BSA,SBH,SBD,SBA
0,E0,10/08/2018,Man United,Leicester,2,1,H,1,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,11/08/2018,Bournemouth,Cardiff,2,0,H,1,0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,11/08/2018,Fulham,Crystal Palace,0,2,A,0,1,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,11/08/2018,Huddersfield,Chelsea,0,3,A,0,2,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,11/08/2018,Newcastle,Tottenham,1,2,A,1,2,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,E0,2010-05-09,Everton,Portsmouth,1,0,H,0,0,D,...,10.00,1.29,5.50,11.00,1.30,5.00,10.00,1.29,5.00,8.50
376,E0,2010-05-09,Hull,Liverpool,0,0,D,0,0,D,...,1.57,6.50,3.75,1.57,6.00,3.75,1.57,6.00,3.75,1.50
377,E0,2010-05-09,Man United,Stoke,4,0,H,2,0,H,...,19.00,1.11,9.00,21.00,1.11,7.50,23.00,1.11,7.50,15.00
378,E0,2010-05-09,West Ham,Man City,1,1,D,1,1,D,...,1.80,3.75,3.60,1.91,4.20,3.60,1.80,4.00,3.40,1.80


In [5]:
def f(x):
    if "/" in x:
        tmp = x.split("/")
        tmp = tmp[2] + "-" + tmp[1] + "-" + tmp[0]
        if len(tmp) == 8:
            return "20" + tmp
        return tmp
    elif len(x) == 8:
        return '20' + x
    else:
        return x
tot_game_df["date"] = tot_game_df["Date"].apply(lambda x : f(x))

In [6]:
data_dir = "../"

In [7]:
world_game = pd.DataFrame()
with fileinput.input(files=data_dir + "worldfootball_crawler/game_worldfootball.jl") as file:
    for line in file:
        conv = json.loads(line)
        world_game = world_game.append(conv, ignore_index=True)
world_game.head()

,season,round,data,time,home_team,away_team,score,home_team_url,away_team_url,goal_player,...,home_team_manager_url,away_team_manager,away_team_manager_url,stadium,stadium_country,stadium_url,refree_name,refree_country,refree_url,num_audience
0,2009/2010,7,26/09/2009,15:00,Birmingham City,Bolton Wanderers,1:2,https://www.worldfootball.net/teams/birmingham...,https://www.worldfootball.net/teams/bolton-wan...,"[Tamir Cohen, Kevin Davies, Kevin Phillips, Ch...",...,https://www.worldfootball.net/player_summary/a...,Gary Megson,https://www.worldfootball.net/player_summary/g...,St Andrew's Ground,Birmingham / England,https://www.worldfootball.net/venues/st-andrew...,Steve Bennett,England,https://www.worldfootball.net/referee_summary/...,28671
1,2009/2010,7,26/09/2009,12:45,Portsmouth FC,Everton FC,0:1,https://www.worldfootball.net/teams/portsmouth...,https://www.worldfootball.net/teams/everton-fc/,"[Louis Saha, John Heitinga]",...,https://www.worldfootball.net/player_summary/p...,David Moyes,https://www.worldfootball.net/player_summary/d...,Fratton Park,Portsmouth / England,https://www.worldfootball.net/venues/fratton-p...,Alan Wiley,England,https://www.worldfootball.net/referee_summary/...,18116
2,2009/2010,5,12/09/2009,15:00,Blackburn Rovers,Wolverhampton Wanderers,3:1,https://www.worldfootball.net/teams/blackburn-...,https://www.worldfootball.net/teams/wolverhamp...,"[El-Hadji Diouf, Jason Roberts, Gaël Givet, Da...",...,https://www.worldfootball.net/player_summary/s...,Mick McCarthy,https://www.worldfootball.net/player_summary/m...,Ewood Park,Blackburn / England,https://www.worldfootball.net/venues/ewood-par...,Steve Bennett,England,https://www.worldfootball.net/referee_summary/...,24845
3,2009/2010,4,29/08/2009,12:45,Chelsea FC,Burnley FC,3:0,https://www.worldfootball.net/teams/chelsea-fc/,https://www.worldfootball.net/teams/burnley-fc/,"[Nicolas Anelka, Didier Drogba, Michael Ballac...",...,https://www.worldfootball.net/player_summary/c...,Owen Coyle,https://www.worldfootball.net/player_summary/o...,Stamford Bridge,London / England,https://www.worldfootball.net/venues/stamford-...,Mark Clattenburg,England,https://www.worldfootball.net/referee_summary/...,40906
4,2009/2010,5,12/09/2009,15:00,Liverpool FC,Burnley FC,4:0,https://www.worldfootball.net/teams/liverpool-fc/,https://www.worldfootball.net/teams/burnley-fc/,"[Yossi Benayoun, Glen Johnson, Dirk Kuyt, Yoss...",...,https://www.worldfootball.net/player_summary/r...,Owen Coyle,https://www.worldfootball.net/player_summary/o...,Anfield,Liverpool / England,https://www.worldfootball.net/venues/anfield-l...,Lee Mason,England,https://www.worldfootball.net/referee_summary/...,43817


In [8]:
len(world_game)

4444

In [9]:
who_game = pd.DataFrame()
with fileinput.input(files=data_dir + "whoscored_crawler/games_whoscored.jl") as file:
    for line in file:
        conv = json.loads(line)
        who_game = who_game.append(conv, ignore_index=True)
who_game.head()

,game_url,season,home_team_formation,home_team_rating,home_team,home_team_url,away_team_formation,away_team_rating,away_team,away_team_url,player_rating,time,date
0,https://www.whoscored.com/Matches/959577/Live/...,2015/2016,4-4-1-1,6.86,West Brom,https://www.whoscored.com/Teams/175/Show/Engla...,4-3-3,6.97,Aston Villa,https://www.whoscored.com/Teams/24/Show/Englan...,"{'11530': {'rating': '7.3', 'shirt_number': '1...",15:00,"Sat, 23-Jan-16"
1,https://www.whoscored.com/Matches/411043/Live/...,2010/2011,4-4-2,7.08,Stoke,https://www.whoscored.com/Teams/96/Show/Englan...,4-5-1,6.74,Aston Villa,https://www.whoscored.com/Teams/24/Show/Englan...,"{'3184': {'rating': '7.5', 'shirt_number': '29...",20:00,"Mon, 13-Sep-10"
2,https://www.whoscored.com/Matches/411264/Live/...,2010/2011,4-4-2,6.72,Blackpool,https://www.whoscored.com/Teams/93/Show/Englan...,4-4-2,7.13,Birmingham,https://www.whoscored.com/Teams/157/Show/Engla...,"{'21050': {'rating': '6.7', 'shirt_number': '2...",20:00,"Tue, 04-Jan-11"
3,https://www.whoscored.com/Matches/1285010/Live...,2018/2019,4-1-4-1,7.54,Leicester,https://www.whoscored.com/Teams/14/Show/Englan...,4-4-2,6.26,Arsenal,https://www.whoscored.com/Teams/13/Show/Englan...,"{'19545': {'rating': '7.3', 'shirt_number': '1...",12:00,"Sun, 28-Apr-19"
4,https://www.whoscored.com/Matches/411389/Live/...,2010/2011,4-4-2,7.03,Man Utd,https://www.whoscored.com/Teams/32/Show/Englan...,4-5-1,6.60,Everton,https://www.whoscored.com/Teams/31/Show/Englan...,"{'3101': {'rating': '7.1', 'shirt_number': '1....",12:45,"Sat, 23-Apr-11"


In [10]:
len(who_game)

4614

In [11]:
month_dict = {"Jan" : "01", "Feb" : "02", "Mar" : "03", "Apr" : "04", "May" : "05", "Jun" : "06", "Jul" : "07", "Aug" : "08", "Sep" : "09", "Oct" : "10", "Nov" : "11", "Dec" : 12}

In [12]:
who_game["date"] = who_game["date"].apply(lambda x : x.split(",")[1].split("-")).apply(lambda x : "20" + str(x[2]) + "-" + str(month_dict[x[1]]) + "-" + str(x[0]).strip())


In [13]:
world_game["date"] = world_game["data"].apply(lambda x : x.split("/"))
world_game["date"] = world_game["date"].apply(lambda x : x[2] + "-" + x[1] + "-" + x[0])

In [14]:
## get team match
team_match = pd.read_csv("../data_/" + "team_match.csv", index_col = 'id')

In [15]:
world_game["home_team_id"] = world_game['home_team_url'].apply(lambda x : team_match[team_match["world_team_url"] == x].index[0])

world_game["away_team_id"] = world_game['away_team_url'].apply(lambda x : team_match[team_match["world_team_url"] == x].index[0])




In [16]:
def f(x):
    tmp = x
    if x == 'https://www.whoscored.com/Teams/175/Show/England-West-Brom':
        tmp = 'https://www.whoscored.com/Teams/175/Show/England-West-Bromwich-Albion'

    def g(x):
        if type(x)== float:
            return None
        else:
            return x.split("/")[4]
    
    return team_match[team_match["who_team_url"].apply(lambda x : g(x)) == tmp.split("/")[4]].index[0]
who_game["home_team_id"] = who_game['home_team_url'].apply(lambda x : f(x))
who_game["away_team_id"] = who_game['away_team_url'].apply(lambda x : f(x))


In [17]:
team_match["team_name"] = team_match["wiki_team_url"].apply(lambda x : x.split("/")[-1])

In [18]:
wiki_name2idx = dict()
wiki_idx2name = dict()
for idx, row in team_match.iterrows():
    wiki_name2idx[row.team_name] = idx
    wiki_idx2name[idx] = row.team_name

In [19]:
Lev = NormalizedLevenshtein().similarity

In [20]:
def f(x):
    tmp = [0,0]
    for name in team_match["team_name"]:
        if Lev(x, name) > 0.3 and Lev(x, name) > tmp[1]:
            tmp[0] = wiki_name2idx[name]
            tmp[1] = Lev(x,name)
    if x == "Wolves":
        return 3
    if x == "QPR":
        return 24
    if x == 'Man City':
        return 7
    if tmp != [0,0]:
        return int(tmp[0])

tot_game_df["home_team_id"] = tot_game_df["HomeTeam"].apply(f)
tot_game_df["away_team_id"] = tot_game_df["AwayTeam"].apply(f)

In [21]:
world_who_game = world_game.merge(who_game, left_on = ["home_team_id", "away_team_id", "date"], right_on = ["home_team_id", "away_team_id", "date"], how = "outer", suffixes = ["_world", "_who"])

In [22]:
all_merged = world_who_game.merge(tot_game_df, left_on = ["home_team_id", "away_team_id", "date"], right_on = ["home_team_id", "away_team_id", "date"], how = "outer", suffixes = ["_world_who", "_structured"])


In [23]:
all_merged.to_csv("../data_/" + "game_join.csv", index = None)

In [24]:
len(tot_game_df)

3420

In [25]:
len(world_who_game)

4829

In [30]:
# Div = League Division
# Date = Match Date (dd/mm/yy)
# HomeTeam = Home Team
# AwayTeam = Away Team
# FTHG = Full Time Home Team Goals
# FTAG = Full Time Away Team Goals
# FTR = Full Time Result (H=Home Win, D=Draw, A=Away Win)
# HTHG = Half Time Home Team Goals
# HTAG = Half Time Away Team Goals
# HTR = Half Time Result (H=Home Win, D=Draw, A=Away Win)

# Match Statistics (where available)
# Attendance = Crowd Attendance
# Referee = Match Referee
# HS = Home Team Shots
# AS = Away Team Shots
# HST = Home Team Shots on Target
# AST = Away Team Shots on Target
# HHW = Home Team Hit Woodwork
# AHW = Away Team Hit Woodwork
# HC = Home Team Corners
# AC = Away Team Corners
# HF = Home Team Fouls Committed
# AF = Away Team Fouls Committed
# HO = Home Team Offsides
# AO = Away Team Offsides
# HY = Home Team Yellow Cards
# AY = Away Team Yellow Cards
# HR = Home Team Red Cards
# AR = Away Team Red Cards
# HBP = Home Team Bookings Points (10 = yellow, 25 = red)
# ABP = Away Team Bookings Points (10 = yellow, 25 = red)

# Key to 1X2 (match) betting odds data:

# B365H = Bet365 home win odds
# B365D = Bet365 draw odds
# B365A = Bet365 away win odds
# BSH = Blue Square home win odds
# BSD = Blue Square draw odds
# BSA = Blue Square away win odds
# BWH = Bet&Win home win odds
# BWD = Bet&Win draw odds
# BWA = Bet&Win away win odds
# GBH = Gamebookers home win odds
# GBD = Gamebookers draw odds
# GBA = Gamebookers away win odds
# IWH = Interwetten home win odds
# IWD = Interwetten draw odds
# IWA = Interwetten away win odds
# LBH = Ladbrokes home win odds
# LBD = Ladbrokes draw odds
# LBA = Ladbrokes away win odds
# PSH = Pinnacle Sports home win odds
# PSD = Pinnacle Sports draw odds
# PSA = Pinnacle Sports away win odds
# SOH = Sporting Odds home win odds
# SOD = Sporting Odds draw odds
# SOA = Sporting Odds away win odds
# SBH = Sportingbet home win odds
# SBD = Sportingbet draw odds
# SBA = Sportingbet away win odds
# SJH = Stan James home win odds
# SJD = Stan James draw odds
# SJA = Stan James away win odds
# SYH = Stanleybet home win odds
# SYD = Stanleybet draw odds
# SYA = Stanleybet away win odds
# VCH = VC Bet home win odds
# VCD = VC Bet draw odds
# VCA = VC Bet away win odds
# WHH = William Hill home win odds
# WHD = William Hill draw odds
# WHA = William Hill away win odds

# Bb1X2 = Number of BetBrain bookmakers used to calculate match odds averages and maximums
# BbMxH = Betbrain maximum home win odds
# BbAvH = Betbrain average home win odds
# BbMxD = Betbrain maximum draw odds
# BbAvD = Betbrain average draw win odds
# BbMxA = Betbrain maximum away win odds
# BbAvA = Betbrain average away win odds



# Key to total goals betting odds:

# BbOU = Number of BetBrain bookmakers used to calculate over/under 2.5 goals (total goals) averages and maximums
# BbMx>2.5 = Betbrain maximum over 2.5 goals
# BbAv>2.5 = Betbrain average over 2.5 goals
# BbMx<2.5 = Betbrain maximum under 2.5 goals
# BbAv<2.5 = Betbrain average under 2.5 goals

# GB>2.5 = Gamebookers over 2.5 goals
# GB<2.5 = Gamebookers under 2.5 goals
# B365>2.5 = Bet365 over 2.5 goals
# B365<2.5 = Bet365 under 2.5 goals


# Key to Asian handicap betting odds:

# BbAH = Number of BetBrain bookmakers used to Asian handicap averages and maximums
# BbAHh = Betbrain size of handicap (home team)
# BbMxAHH = Betbrain maximum Asian handicap home team odds
# BbAvAHH = Betbrain average Asian handicap home team odds
# BbMxAHA = Betbrain maximum Asian handicap away team odds
# BbAvAHA = Betbrain average Asian handicap away team odds

# GBAHH = Gamebookers Asian handicap home team odds
# GBAHA = Gamebookers Asian handicap away team odds
# GBAH = Gamebookers size of handicap (home team)
# LBAHH = Ladbrokes Asian handicap home team odds
# LBAHA = Ladbrokes Asian handicap away team odds
# LBAH = Ladbrokes size of handicap (home team)
# B365AHH = Bet365 Asian handicap home team odds
# B365AHA = Bet365 Asian handicap away team odds
# B365AH = Bet365 size of handicap (home team)

# Football-Data would like to acknowledge the following sources which have been utilised in the compilation of Football-Data's results and odds files.

# Historical results:
# International Soccer Server - http://sunsite.tut.fi/rec/riku/soccer.html
# European Football - http://www.eurofootball.be/
# RSSSF Archive - http://www.rsssf.com/

# Current results (full time, half time)
# TBWSport - http://www.tbwsport.com
# Livescore- http://www.livescore.com

# Match statistics
# Sportinglife, ESPN Soccer, Bundesliga.de, Gazzetta.it and Football.fr

# Bookmakers betting odds
# Betbrain - http://www.betbrain.com
# Betbase - http://www.betbase.info

# Betting odds for weekend games are collected Friday afternoons, and on Tuesday afternoons for midweek games.